In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sshikamaru/lyme-disease-rashes
!unzip lyme-disease-rashes.zip

Streaming output truncated to the last 5000 lines.
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash168.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash169.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash17.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash170.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash171.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash172.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash173.jpg  
  inflating: Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train/Negative/spider bite rash174.jpg 

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.8 MB/s eta 0:00:00


CROSS VALIDATION!!!!!

In [ ]:
import numpy as np
import os
import cv2
import glob
from skimage.feature import local_binary_pattern
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Input
from keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from deap import base, creator, tools, algorithms
from sklearn.model_selection import KFold
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Paths to the dataset folders
train_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train"
test_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/test"
val_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/val"

# Initialize the base VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
cnn_model = Model(inputs=base_model.input, outputs=x)

# Preprocess image for VGG16
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize image
    return image

# Extract LBP features
def extract_lbp_features(image):
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype('float')
    hist /= hist.sum()
    return hist

# Genetic algorithm functions
def initialize_population(image, pop_size):
    population = []
    for _ in range(pop_size):
        roi = np.zeros_like(image)
        height, width = image.shape
        max_intensity = np.max(image)
        stable_intensity = np.mean(image)

        while True:
            x1, y1 = random.randint(0, width-1), random.randint(0, height-1)
            x2, y2 = random.randint(0, width-1), random.randint(0, height-1)
            roi[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] = 255
            if np.mean(image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]) < stable_intensity:
                break

        individual = creator.Individual(roi)
        population.append(individual)
    return population

def evaluate_individual(individual, image):
    masked_image = cv2.bitwise_and(image, image, mask=individual)
    return np.sum(masked_image),

def mutate_individual(individual, image):
    height, width = individual.shape
    max_intensity = np.max(image)
    stable_intensity = np.mean(image)

    while True:
        x1, y1 = random.randint(0, width-1), random.randint(0, height-1)
        x2, y2 = random.randint(0, width-1), random.randint(0, height-1)
        individual[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] = 255
        if np.mean(image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]) < stable_intensity:
            break

    return individual,

def crossover_individual(ind1, ind2):
    height, width = ind1.shape
    cx, cy = random.randint(0, width-1), random.randint(0, height-1)

    temp1 = np.copy(ind1[cx:, :])
    temp2 = np.copy(ind2[cx:, :])

    ind1[cx:, :] = temp2
    ind2[cx:, :] = temp1

    return ind1, ind2

def genetic_algorithm(image, pop_size=10, generations=10):
    # Recreate the Fitness and Individual classes to avoid warnings
    if "FitnessMax" in creator.__dict__:
        del creator.FitnessMax
    if "Individual" in creator.__dict__:
        del creator.Individual

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("population", initialize_population, image, pop_size)
    toolbox.register("evaluate", evaluate_individual, image=image)
    toolbox.register("mate", crossover_individual)
    toolbox.register("mutate", mutate_individual, image=image)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population()

    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=generations, verbose=False)

    best_individual = tools.selBest(population, 1)[0]
    return best_individual

# Function to load and extract features from the dataset
def load_and_extract_features(path):
    features = []
    labels = []
    for label in ['Positive', 'Negative']:
        for file in glob.glob(os.path.join(path, label, '*.jpg')):
            image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue

            best_roi = genetic_algorithm(image)
            masked_image = cv2.bitwise_and(image, image, mask=best_roi)

            lbp_features = extract_lbp_features(masked_image)

            # Convert masked image to 3 channels by stacking the grayscale image
            masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_GRAY2RGB)
            preprocessed_masked_image = preprocess_image(masked_image_rgb)

            cnn_features = cnn_model.predict(preprocessed_masked_image)

            combined_features = np.concatenate((cnn_features.flatten(), lbp_features))
            features.append(combined_features)
            labels.append(1 if label == 'Positive' else 0)

    return np.array(features), np.array(labels)

# Load and extract features from the datasets
X, y = load_and_extract_features(train_path)

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
results = []

for train_index, val_index in kf.split(X):
    print(f'Training fold {fold_no}...')
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Create and compile the combined model
    def create_combined_model(input_dim):
        input_layer = Input(shape=(input_dim,))
        x = Dense(128, activation='relu')(input_layer)
        x = Dense(64, activation='relu')(x)
        output_layer = Dense(1, activation='sigmoid')(x)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    input_dim = X_train.shape[1]
    model = create_combined_model(input_dim)

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

    # Evaluate the model
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f'Validation loss: {val_loss}, Validation accuracy: {val_acc}')
    results.append(val_acc)
    fold_no += 1

# Report average performance
print(f'Average validation accuracy: {np.mean(results)}')

# Evaluate on test data
X_test, y_test = load_and_extract_features(test_path)
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred))


1/1 [==============================] - 0s 81ms/step


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 74ms/step
Training fold 1...
Epoch 1/50
102/102 [==============================] - 2s 13ms/step - loss: 0.5080 - accuracy: 0.8152 - val_loss: 0.4837 - val_accuracy: 0.8037
Epoch 2/50
102/102 [==============================] - 1s 10ms/step - loss: 0.4094 - accuracy: 0.8273 - val_loss: 0.6514 - val_accuracy: 0.8000
Epoch 3/50
102/102 [==============================] - 1s 10ms/step - loss: 0.3717 - accuracy: 0.8504 - val_loss: 0.5018 - val_accuracy: 0.8062
Epoch 4/50
102/102 [==============================] - 1s 10ms/step - loss: 0.2865 - accuracy: 0.8810 - val_loss: 0.5296 - val_accuracy: 0.7926
Epoch 5/50
102/102 [==============================] - 1s 10ms/step - loss: 0.2404 - accuracy: 0.9030 - val_loss: 0.5812 - val_accuracy: 0.8012
Epoch 6/50
102/102 [==============================] - 1s 10ms/step - loss: 0.1920 - accuracy: 0.9271 - val_loss: 0.6495 - val_accuracy: 0.7840
Epoch 7/50
102/102 [

In [ ]:
import numpy as np
import os
import cv2
import glob
from skimage.feature import local_binary_pattern
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Input
from keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import classification_report
from deap import base, creator, tools, algorithms
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Paths to the dataset folders
train_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train"
test_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/test"
val_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/val"

# Initialize the base VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
cnn_model = Model(inputs=base_model.input, outputs=x)

# Preprocess image for VGG16
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize image
    return image

# Extract LBP features
def extract_lbp_features(image):
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype('float')
    hist /= hist.sum()
    return hist

# Genetic algorithm functions
def initialize_population(image, pop_size):
    population = []
    for _ in range(pop_size):
        roi = np.zeros_like(image)
        height, width = image.shape
        max_intensity = np.max(image)
        stable_intensity = np.mean(image)

        while True:
            x1, y1 = random.randint(0, width-1), random.randint(0, height-1)
            x2, y2 = random.randint(0, width-1), random.randint(0, height-1)
            roi[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] = 255
            if np.mean(image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]) < stable_intensity:
                break

        individual = creator.Individual(roi)
        population.append(individual)
    return population

def evaluate_individual(individual, image):
    masked_image = cv2.bitwise_and(image, image, mask=individual)
    return np.sum(masked_image),

def mutate_individual(individual, image):
    height, width = individual.shape
    max_intensity = np.max(image)
    stable_intensity = np.mean(image)

    while True:
        x1, y1 = random.randint(0, width-1), random.randint(0, height-1)
        x2, y2 = random.randint(0, width-1), random.randint(0, height-1)
        individual[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] = 255
        if np.mean(image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]) < stable_intensity:
            break

    return individual,

def crossover_individual(ind1, ind2):
    height, width = ind1.shape
    cx, cy = random.randint(0, width-1), random.randint(0, height-1)

    temp1 = np.copy(ind1[cx:, :])
    temp2 = np.copy(ind2[cx:, :])

    ind1[cx:, :] = temp2
    ind2[cx:, :] = temp1

    return ind1, ind2

def genetic_algorithm(image, pop_size=10, generations=10):
    # Recreate the Fitness and Individual classes to avoid warnings
    if "FitnessMax" in creator.__dict__:
        del creator.FitnessMax
    if "Individual" in creator.__dict__:
        del creator.Individual

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("population", initialize_population, image, pop_size)
    toolbox.register("evaluate", evaluate_individual, image=image)
    toolbox.register("mate", crossover_individual)
    toolbox.register("mutate", mutate_individual, image=image)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population()

    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=generations, verbose=False)

    best_individual = tools.selBest(population, 1)[0]
    return best_individual

# Function to load and extract features from the dataset
def load_and_extract_features(path):
    features = []
    labels = []
    for label in ['Positive', 'Negative']:
        for file in glob.glob(os.path.join(path, label, '*.jpg')):
            image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue

            best_roi = genetic_algorithm(image)
            masked_image = cv2.bitwise_and(image, image, mask=best_roi)

            lbp_features = extract_lbp_features(masked_image)

            # Convert masked image to 3 channels by stacking the grayscale image
            masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_GRAY2RGB)
            preprocessed_masked_image = preprocess_image(masked_image_rgb)

            cnn_features = cnn_model.predict(preprocessed_masked_image)

            combined_features = np.concatenate((cnn_features.flatten(), lbp_features))
            features.append(combined_features)
            labels.append(1 if label == 'Positive' else 0)

    return np.array(features), np.array(labels)

# Load and extract features from the datasets
X_train, y_train = load_and_extract_features(train_path)
X_test, y_test = load_and_extract_features(test_path)
X_val, y_val = load_and_extract_features(val_path)

# Create and compile the combined model with reduced complexity
def create_combined_model(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(64, activation='relu')(input_layer)
    x = Dense(32, activation='relu')(x)
    output_layer = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Combine LBP and CNN features for training
input_dim = X_train.shape[1]
model = create_combined_model(input_dim)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred))


1/1 [==============================] - 0s 73ms/step


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 73ms/step
Epoch 1/50
127/127 [==============================] - 2s 8ms/step - loss: 0.5332 - accuracy: 0.8040 - val_loss: 0.5334 - val_accuracy: 0.7846
Epoch 2/50
127/127 [==============================] - 1s 6ms/step - loss: 0.4544 - accuracy: 0.8154 - val_loss: 0.4585 - val_accuracy: 0.8261
Epoch 3/50
127/127 [==============================] - 1s 6ms/step - loss: 0.4032 - accuracy: 0.8334 - val_loss: 0.4565 - val_accuracy: 0.8202
Epoch 4/50
127/127 [==============================] - 1s 6ms/step - loss: 0.3557 - accuracy: 0.8537 - val_loss: 0.4504 - val_accuracy: 0.8182
Epoch 5/50
127/127 [==============================] - 1s 6ms/step - loss: 0.3278 - accuracy: 0.8631 - val_loss: 0.5520 - val_accuracy: 0.8202
Epoch 6/50
127/127 [==============================] - 1s 7ms/step - loss: 0.2861 - accuracy: 0.8930 - val_loss: 0.4699 - val_accuracy: 0.8202
Epoch 7/50
127/127 [=========================

In [ ]:
import numpy as np
import os
import cv2
import glob
from skimage.feature import local_binary_pattern
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Input
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
import random

# Paths to the dataset folders
train_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train"
test_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/test"
val_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/val"

# Initialize the base VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
cnn_model = Model(inputs=base_model.input, outputs=x)

# Preprocess image for VGG16
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize image
    return image

# Extract LBP features
def extract_lbp_features(image):
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype('float')
    hist /= hist.sum()
    return hist

# Image denoising
def denoise_image(image):
    return cv2.medianBlur(image, 3)

# Genetic algorithm functions
def initialize_population(image, pop_size):
    population = []
    for _ in range(pop_size):
        roi = np.zeros_like(image)
        height, width = image.shape
        max_intensity = np.max(image)
        stable_intensity = np.mean(image)

        while True:
            x1, y1 = random.randint(0, width-1), random.randint(0, height-1)
            x2, y2 = random.randint(0, width-1), random.randint(0, height-1)
            roi[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] = 255
            if np.mean(image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]) < stable_intensity:
                break

        individual = creator.Individual(roi)
        population.append(individual)
    return population

def evaluate_individual(individual, image):
    masked_image = cv2.bitwise_and(image, image, mask=individual)
    return np.sum(masked_image),

def mutate_individual(individual, image):
    height, width = individual.shape
    max_intensity = np.max(image)
    stable_intensity = np.mean(image)

    while True:
        x1, y1 = random.randint(0, width-1), random.randint(0, height-1)
        x2, y2 = random.randint(0, width-1), random.randint(0, height-1)
        individual[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] = 255
        if np.mean(image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]) < stable_intensity:
            break

    return individual,

def crossover_individual(ind1, ind2):
    height, width = ind1.shape
    cx, cy = random.randint(0, width-1), random.randint(0, height-1)

    temp1 = np.copy(ind1[cx:, :])
    temp2 = np.copy(ind2[cx:, :])

    ind1[cx:, :] = temp2
    ind2[cx:, :] = temp1

    return ind1, ind2

def genetic_algorithm(image, pop_size=10, generations=10):
    # Recreate the Fitness and Individual classes to avoid warnings
    if "FitnessMax" in creator.__dict__:
        del creator.FitnessMax
    if "Individual" in creator.__dict__:
        del creator.Individual

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("population", initialize_population, image, pop_size)
    toolbox.register("evaluate", evaluate_individual, image=image)
    toolbox.register("mate", crossover_individual)
    toolbox.register("mutate", mutate_individual, image=image)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population()
aSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=generations, verbose=False)

    best_individual = tools.selBest(population, 1)[0]
    return best_individual

# Function to load and extract features from the dataset
def load_and_extract_features(path):
    features = []
    labels = []
    for label in ['Positive', 'Negative']:
        for file in glob.glob(os.path.join(path, label, '*.jpg')):
            image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue

            # Apply denoising
            image = denoise_image(image)

            best_roi = genetic_algorithm(image)
            masked_image = cv2.bitwise_and(image, image, mask=best_roi)

            lbp_features = extract_lbp_features(masked_image)

            # Convert masked image to 3 channels by stacking the grayscale image
            masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_GRAY2RGB)
            preprocessed_masked_image = preprocess_image(masked_image_rgb)

            cnn_features = cnn_model.predict(preprocessed_masked_image)

            combined_features = np.concatenate((cnn_features.flatten(), lbp_features))
            features.append(combined_features)
            labels.append(1 if label == 'Positive' else 0)

    return np.array(features), np.array(labels)

# Load and extract features from the datasets
X_train, y_train = load_and_extract_features(train_path)
X_test, y_test = load_and_extract_features(test_path)
X_val, y_val = load_and_extract_features(val_path)

# Create and compile the combined model
def create_combined_model(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(128, activation='relu', kernel_regularizer='l2')(input_layer)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu', kernel_regularizer='l2')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Combine LBP and CNN features for training
input_dim = X_train.shape[1]
model = create_combined_model(input_dim)

# Implement early stopping
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred))



/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 545ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 548ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 531ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 938ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 929ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 982ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 944ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 526ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 545ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 596ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator


In [ ]:
import numpy as np
import os
import cv2
import glob
from skimage.feature import local_binary_pattern
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Input
#from keras.preprocessing.image import img_to_array, ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
import random
from keras.callbacks import EarlyStopping

# Paths to the dataset folders
train_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/train"
test_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/test"
val_path = "/content/Lyme Full Statified/Lyme Full Statified/DataSplit/image_folder 2/val"

# Initialize the base VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
cnn_model = Model(inputs=base_model.input, outputs=x)

# Preprocess image for VGG16
def preprocess_image(image):
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize image
    return image

# Extract LBP features
def extract_lbp_features(image):
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype('float')
    hist /= hist.sum()
    return hist

# Image denoising
def denoise_image(image):
    return cv2.medianBlur(image, 3)

# Genetic algorithm functions
def initialize_population(image, pop_size):
    population = []
    for _ in range(pop_size):
        roi = np.zeros_like(image)
        height, width = image.shape
        max_intensity = np.max(image)
        stable_intensity = np.mean(image)

        while True:
            x1, y1 = random.randint(0, width-1), random.randint(0, height-1)
            x2, y2 = random.randint(0, width-1), random.randint(0, height-1)
            roi[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] = 255
            if np.mean(image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]) < stable_intensity:
                break

        individual = creator.Individual(roi)
        population.append(individual)
    return population

def evaluate_individual(individual, image):
    masked_image = cv2.bitwise_and(image, image, mask=individual)
    return np.sum(masked_image),

def mutate_individual(individual, image):
    height, width = individual.shape
    max_intensity = np.max(image)
    stable_intensity = np.mean(image)

    while True:
        x1, y1 = random.randint(0, width-1), random.randint(0, height-1)
        x2, y2 = random.randint(0, width-1), random.randint(0, height-1)
        individual[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)] = 255
        if np.mean(image[min(y1, y2):max(y1, y2), min(x1, x2):max(x1, x2)]) < stable_intensity:
            break

    return individual,

def crossover_individual(ind1, ind2):
    height, width = ind1.shape
    cx, cy = random.randint(0, width-1), random.randint(0, height-1)

    temp1 = np.copy(ind1[cx:, :])
    temp2 = np.copy(ind2[cx:, :])

    ind1[cx:, :] = temp2
    ind2[cx:, :] = temp1

    return ind1, ind2

def genetic_algorithm(image, pop_size=10, generations=10):
    # Recreate the Fitness and Individual classes to avoid warnings
    if "FitnessMax" in creator.__dict__:
        del creator.FitnessMax
    if "Individual" in creator.__dict__:
        del creator.Individual

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("population", initialize_population, image, pop_size)
    toolbox.register("evaluate", evaluate_individual, image=image)
    toolbox.register("mate", crossover_individual)
    toolbox.register("mutate", mutate_individual, image=image)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population()
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=generations, verbose=False)

    best_individual = tools.selBest(population, 1)[0]
    return best_individual

# Function to load and extract features from the dataset
def load_and_extract_features(path, augment=False):
    features = []
    labels = []
    datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest') if augment else None
    for label in ['Positive', 'Negative']:
        for file in glob.glob(os.path.join(path, label, '*.jpg')):
            image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue

            # Apply denoising
            image = denoise_image(image)

            best_roi = genetic_algorithm(image)
            masked_image = cv2.bitwise_and(image, image, mask=best_roi)

            lbp_features = extract_lbp_features(masked_image)

            # Convert masked image to 3 channels by stacking the grayscale image
            masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_GRAY2RGB)
            preprocessed_masked_image = preprocess_image(masked_image_rgb)

            cnn_features = cnn_model.predict(preprocessed_masked_image)

            combined_features = np.concatenate((cnn_features.flatten(), lbp_features))
            features.append(combined_features)
            labels.append(1 if label == 'Positive' else 0)

            if augment:
                image = np.expand_dims(masked_image_rgb, axis=0)
                i = 0
                for batch in datagen.flow(image, batch_size=1):
                    augmented_image = cv2.cvtColor(batch[0].astype('float32'), cv2.COLOR_RGB2GRAY)
                    lbp_features_aug = extract_lbp_features(augmented_image)
                    preprocessed_aug_image = preprocess_image(batch[0])
                    cnn_features_aug = cnn_model.predict(preprocessed_aug_image)
                    combined_features_aug = np.concatenate((cnn_features_aug.flatten(), lbp_features_aug))
                    features.append(combined_features_aug)
                    labels.append(1 if label == 'Positive' else 0)
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per original image
                        break

    return np.array(features), np.array(labels)

# Load and extract features from the datasets
X_train, y_train = load_and_extract_features(train_path, augment=True)
X_test, y_test = load_and_extract_features(test_path)
X_val, y_val = load_and_extract_features(val_path)

# Create and compile the combined model
def create_combined_model(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(128, activation='relu', kernel_regularizer='l2')(input_layer)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu', kernel_regularizer='l2')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Combine LBP and CNN features for training
input_dim = X_train.shape[1]
model = create_combined_model(input_dim)

# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 858ms/step


/usr/local/lib/python3.10/dist-packages/skimage/feature/texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 821ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 901ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 509ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 745ms/step
1/1 ━━━━━━━━━

In [ ]:

import cv2
import numpy as np

def detect_lyme_disease_rash(image):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for red color in HSV
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    
    # Threshold the HSV image to get only red colors
    mask1 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])
    
    # Combine masks
    red_mask = mask1 | mask2
    
    # Bitwise-AND mask and original image
    result = cv2.bitwise_and(image, image, mask=red_mask)
    
    return result

# Example usage
image_path = 'path_to_image.jpg'
image = cv2.imread(image_path)
detected_rash = detect_lyme_disease_rash(image)
cv2.imshow('Detected Rash', detected_rash)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:

def initialize_population(detected_rash, population_size=50):
    # Find contours in the detected rash image
    contours, _ = cv2.findContours(detected_rash, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Extract bounding boxes
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]
    
    # Initialize population with bounding boxes
    population = []
    for _ in range(population_size):
        for box in bounding_boxes:
            x, y, w, h = box
            population.append((x, y, w, h))
    
    return population

# Example usage
population = initialize_population(detected_rash)


In [ ]:

def fitness_function(candidate, image):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    
    # Feature extraction
    mean_color = np.mean(roi, axis=(0, 1))
    texture = cv2.Laplacian(cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY), cv2.CV_64F).var()
    
    # Fitness calculation
    red_intensity = mean_color[2] - (mean_color[0] + mean_color[1]) / 2  # Red channel intensity
    fitness = red_intensity + texture
    
    return fitness

# Example usage
candidate = population[0]
fitness = fitness_function(candidate, image)
print(f'Fitness: {fitness}')


In [ ]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'path_to_train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    'path_to_validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)
